In [25]:
import pandas as pd
import numpy as np
import os
import re
import jieba
import jieba.posseg as pseg
import matplotlib.pyplot as plt

In [26]:
data = pd.read_csv('lk_reviews_720.csv',lineterminator='\n')

In [37]:
jieba.load_userdict("./user_dict/dic_marvel_movie.txt")


#jieba.add_word("")  

# stop_words_path='stopwords'
# stopwords1 = [line.rstrip() for line in open(os.path.join(stop_words_path, '中文停用词表.txt'), 'r', encoding='utf-8')]
# stopwords2 = [line.rstrip() for line in open(os.path.join(stop_words_path, '哈工大停用词表.txt'), 'r', encoding='utf-8')]
# stopwords3 = [line.rstrip() for line in open(os.path.join(stop_words_path, '四川大学机器智能实验室停用词库.txt'), 'r', encoding='utf-8')]
# stopwords4 = [line.rstrip() for line in open(os.path.join(stop_words_path, '百度停用词表.txt'), 'r', encoding='utf-8')]







# try to use pkuseg which has a higher accuracy but much slower than jieba

def proc_text(raw_line):
    """
        process texts
        return words
        Use pkuseg to split the word. take longer time than jieba but more accurate
    """

     1. filter and keep Chinese characters only
    
    filter_pattern = re.compile('[^\u4E00-\u9FD5]+')
    chinese_only = filter_pattern.sub('', raw_line)

     2. segmentation with Pkuseg and part-of-speech tagging
    #detailed explaination for pkuseg: https://github.com/lancopku/pkuseg-python
    
    seg = pkuseg.pkuseg() # use the default model, and mark postag
    word_list = seg.cut(chinese_only)
    
     3. delete stopwords and 
     only keep verbs,adjectives, adverbs, nouns, names, idioms, numbers, and places
    
    used_flags = ['v', 'a', 'ad','n','nr','i','m','ns']
    meaninful_words = []
    for word in word_list:
        if word not in stopwords:
            meaninful_words.append(word)
    return ' '.join(meaninful_words)

In [38]:
 #  covert stop words txt to python dictionary {} to increase the efficiency
stopwords = {}
fstop = open('./user_dict/user_stopword.txt', 'r')
for eachWord in fstop:
    stopwords[eachWord.strip()] = eachWord.strip()
fstop.close()

In [1]:
data

In [40]:
def proc_text(raw_line):
    """
        process texts
        return words
    """

    #  filter and keep Chinese characters only
    filter_pattern = re.compile('[^\u4E00-\u9FD5]+')
    chinese_only = filter_pattern.sub('', str(raw_line))

    #  segmentation with jieba and part-of-speech tagging
    word_list = pseg.cut(chinese_only)

    #  delete stopwords and 
    # only keep verbs,adjectives, adverbs, nouns, names, idioms, numbers, and places
    used_flags = ['v', 'a', 'd','ad','n','nr','i','m','ns']
    meaninful_words = []
    for word,flag in word_list:
        if (word not in stopwords) and (flag in used_flags):
            meaninful_words.append(word)
    return ' '.join(meaninful_words)

In [41]:
data['words'] = data['Content'].apply(proc_text)

In [42]:
seg_rows=[]
for i in range(0,len(data)):
    rowi=data['words'][i]
    seg_rows.append(rowi)
seg_rows=[item.split() for item in seg_rows]
seg_rows=[item for sublist in seg_rows for item in sublist]

In [3]:
seg_rows

In [44]:
gram_list= list(set(seg_rows))
gram_dict= {key:0 for key in gram_list if len(key) > 1}
#count the occurence
for key in gram_dict:
    for cont in data['Content']:
        try:
            if key in cont:
                gram_dict[key] += 1
        except:
            pass

In [53]:
#customization 2
#grouping words that mean the same thing together
group1=[]
group2 = []
group3 = []
group4 = []
group5 = []
group6 = []
group7 = []
for k in list(gram_dict):
    if k in group1:

        gram_dict[]+=gram_dict[k]
        gram_dict.pop(k, None)
    elif k in group2:
        gram_dict[]+=gram_dict[k]
        gram_dict.pop(k, None)
    elif k in group3:
        gram_dict[]+=gram_dict[k]
        gram_dict.pop(k, None)
    elif k in group4:
        gram_dict[]+=gram_dict[k]
        gram_dict.pop(k, None)
    elif k in group5:
        gram_dict[]+=gram_dict[k]
        gram_dict.pop(k, None)
    else:
        pass



In [54]:
ngram_df= pd.DataFrame(list(gram_dict.items()), columns=['N_Gram','Frequency'])

In [7]:
ngram_df.sort_values(by='Frequency',ascending=False)

In [56]:
ngram_sorted=ngram_df.sort_values(by='Frequency',ascending=False)

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)
from wordcloud import WordCloud

wordcloud=WordCloud(font_path="SimHei.ttf",background_color="white",max_font_size=80)
word_frequence = {x[0]:x[1] for x in ngram_sorted.head(100).values}
wordcloud=wordcloud.fit_words(word_frequence)
plt.imshow(wordcloud)

In [9]:
ngram_sorted.to_csv('review_word_count.csv')